[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sparks-baird/self-driving-lab-demo/blob/main/notebooks/4.1-paho-mqtt-colab-sdl-demo-search.ipynb)

# Optimization Comparison using Internet of Things-style Communication

The other device could be your laptop, a Google Colab script, etc.

In [7]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False  

In [8]:
if IN_COLAB:
  %pip install self-driving-lab-demo paho-mqtt

## Pico ID

Load a unique identifier for your Pico device from a `pico_id_secret.py` file that you store in your Google Drive (see below) or current directory (`.`), or specify it manually by setting the `PICO_ID` variable in the form field. If you don't know what the ID is, you can get it by running the following in the Pico REPL (i.e. MicroPython shell, e.g. in Thonny):
```python
from machine import unique_id
from ubinascii import hexlify
my_id = hexlify(unique_id()).decode()
print(my_id)
```

For your convenience, this is done already in [`main.py`](https://github.com/sparks-baird/self-driving-lab-demo/blob/06ededd312087a815b5c0ac8098e82db9c5730c8/src/public_mqtt_sdl_demo/main.py) so you need only run this file once the Pico W has been properly set up and it will be displayed in the terminal.

The file named `pico_id_secret.py` placed directly in your `MyDrive` folder or in your current directory would look like the following:
```python
PICO_ID = "a123b1234azed"
```

In [9]:
#@markdown No need to enter this manually if you've already saved the appropriate file to your Google Drive. Just run this cell and the hidden code will take care of it.
import sys
secret_name = "PICO_ID"
if IN_COLAB:
  try:
    from google.colab import drive
    drive.mount('/content/drive')
    sys.path.append('/content/drive/MyDrive/')
    from pico_id_secret import PICO_ID
  except Exception as e:
    print(e)
    PICO_ID = "test" #@param {type:"string"}
    if PICO_ID == "":
      print(f"Couldn't load {secret_name}, and user-input {secret_name} is also empty.")
    else:
      print(f"defaulting to user-input PICO_ID: {PICO_ID}")
else:
  from public_mqtt_sdl_demo.secrets import PICO_ID

In [10]:
from self_driving_lab_demo import SelfDrivingLabDemo, mqtt_observe_sensor_data

sdl = SelfDrivingLabDemo(
    autoload=True,
    observe_sensor_data_fn=mqtt_observe_sensor_data,
    observe_sensor_data_kwargs=dict(pico_id=PICO_ID),
)
sdl.clear()

We've gone through enough boilerplate code and comparisons by this point that it's
prudent to define and import helper functions based on content from previous notebooks
(e.g. [`3.1-random-vs-grid-vs-bayesian.ipynb`](3.1-random-vs-grid-vs-bayesian.ipynb))

In [11]:
sdl.evaluate(10, 10, 10)

{'ch470': 4082,
 'ch670': 2155,
 'ch550': 1319,
 'ch410': 629,
 'ch440': 1778,
 '_input_message': {'_session_id': '9c2b8629-06cf-47ce-bbb9-5b571af968e0',
  'R': 10,
  'B': 10,
  'G': 10,
  '_experiment_id': 'b0e195bf-beff-4db7-89f7-d9e5dd381d94'},
 'ch583': 1457,
 'ch510': 1462,
 'ch620': 1873,
 'mae': 1995.875,
 'rmse': 2872.929667604134,
 'frechet': 3887.8946744992977}

In [12]:
num_iter = 64

from self_driving_lab_demo.utils.search import (
    grid_search,
    random_search,
    ax_bayesian_optimization,
)

grid, grid_data = grid_search(sdl, num_iter)
random_inputs, random_data = random_search(sdl, num_iter)
best_parameters, values, experiment, model = ax_bayesian_optimization(sdl, num_iter)

[INFO 09-12 13:13:04] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter R. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-12 13:13:04] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter G. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-12 13:13:04] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter B. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-12 13:13:04] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='R', parameter_type=INT, range=[0, 89]), RangeParameter(name='G', parameter_type=INT, range=[0, 89]), RangeParameter(name='B', parameter_type=INT, r